In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('LinearRegression').getOrCreate()

In [2]:
df = spark.read.csv('data/cruise_ship_info.csv', inferSchema=True, header=True)

In [3]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [5]:
for ship in df.head(5):
    print(ship)

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)
Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)
Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)
Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)
Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)


In [6]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [7]:
from pyspark.ml.feature import StringIndexer

In [8]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='cruise_cat')
indexed = indexer.fit(df).transform(df)

for row in indexed.head(5):
    print(row)

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)
Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)
Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)
Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0)
Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)


In [11]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [14]:
assembler = VectorAssembler(
    inputCols=[
        'Tonnage',
        'passengers',
        'length',
        'cabins',
        'passenger_density',
        'cruise_cat'
    ],
    outputCol='features'
)

output = assembler.transform(indexed)

In [15]:
output.select(['features', 'crew']).show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[30.2769999999999...|3.55|
|[30.2769999999999...|3.55|
|[47.262,14.86,7.2...| 6.7|
|[110.0,29.74,9.53...|19.1|
|[101.353,26.42,8....|10.0|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.56,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[110.238999999999...|11.5|
|[110.0,29.74,9.51...|11.6|
|[46.052,14.52,7.2...| 6.6|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[86.0,21.24,9.63,...| 9.3|
|[110.0,29.74,9.51...|11.6|
|[88.5,21.24,9.63,...|10.3|
|[70.367,20.52,8.5...| 9.2|
|[88.5,21.24,9.63,...| 9.3|
|[70.367,20.52,8.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [16]:
final_data = output.select(['features', 'crew'])
training, test = final_data.randomSplit([0.7, 0.3])

training.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              117|
|   mean|7.802905982905983|
| stddev|3.682419336810934|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [17]:
test.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               41|
|   mean|7.769268292682926|
| stddev|2.975166709819722|
|    min|              0.6|
|    max|             13.6|
+-------+-----------------+



In [18]:
from pyspark.ml.regression import LinearRegression

model = LinearRegression(labelCol='crew').fit(training)

In [19]:
results = model.evaluate(test)

In [20]:
results.rootMeanSquaredError

0.6221932179127303

In [21]:
results.r2

0.9551717485501511

In [22]:
from pyspark.sql.functions import corr

In [23]:
df.select(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [24]:
df.select(corr('crew', 'cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

